# 8.2 文本预处理
文本预处理的常见步骤：
- 将文本作为字符串加载到内存中（转字符串）
- 将字符串拆分为词元(token)（拆字符串）
- 建立一个词表(vocabulary)，将拆分的词元映射到数字索引（放词元）
- 将文本转换为数字索引序列，以方便模型操作

In [1]:
import collections
import re
from d2l import torch as d2l

## 8.2.1 读取数据集
读取时光机器数据集。以下的函数将数据集读取到由多个文本本行组成的列表中，每个文本行都是一个字符串

In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine(): #@save
    '''将时间机器数据集加载到文本行的列表中'''
    with open(d2l.download("time_machine"),'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数：{len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数：3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 8.2.2 词元化
下面的tokenize函数将文本行列表(lines)作为输入，lines中的每个元素是一个文本序列（如一个文本行）

每个文本序列又被拆分为一个词元列表(token)，词元(token)是文本的基本单位

最后，返回一个由词元列表组成的列表，其中每个词元都是一个字符串

In [3]:
def tokenize(lines,token='word'): #@save
    '''将文本行拆分为单词或字符次元'''
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 8.2.3 词表
词表(vocabulary)将字符串类型的词元映射到0开始的数字索引中。

语料库(corpus)是将训练集中的所有文档合并在一起后，对它们的唯一词元进行统计后得到的统计结果；然后根据每个唯一词元出现的频率，为其分配一个数字索引

In [5]:
class Vocab: #@save
    '''文本词表'''
    def __init__(self,token=None,min_freq=0,reserved_tokens=None):
        '''
        :param token: 词元列表，可以是一维或二维列表
        :param min_freq: 最小频率阈值，低于该频率的词元将被忽略
        :param reserved_tokens: 保留词元列表，这些词元会被优先添加到词表中
        '''
        if token is None:
            token = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(),key=lambda x:x[1],
                                   reverse=True)
        # 未知词元的索引为0
        ## 初始化idx_to_token列表，将未知词元<unk>和保留词元添加到列表中（列表的拼接）
        self.idx_to_token = ['<unk>'] + reserved_tokens
        ## 初始化token_to_idx字典，将词元映射到对应的索引
        self.token_to_idx = {token:idx
                             for idx,token in enumerate(self.idx_to_token)}
        for token,freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        '''
        :fuction: 返回词表的大小，即词元的数量
        :return: 返回idx_to_token列表的长度
        '''
        return len(self.idx_to_token)

    def __getitem__(self,tokens):
        '''
        :fuction: 将词元或词元列表转换为对应的索引或索引列表
        :param tokens: 单个词元或词元列表
        :return: 索引或索引列表
        '''
        if not isinstance(tokens,(list,tuple)): # 如果不是列表或元组，则转换为索引
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self,indices):
        '''
        :function: 将索引或索引列表转换为对应的词元或词元列表
        :param indices: 单个索引或索引列表
        :return: 索引对应的词元（单个或列表形式）
        '''
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): # 未知词元的索引为0
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens): #@save
    '''统计词元出现的频率'''
    # 这里的tokens是一维列表或二维列表
    if len(tokens) == 0 or isinstance(tokens[0],list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

有关函数count_corpus中嵌套的列表推导式解释：
- 作用：将一个二维列表“展平”成一个一维列表
- 在列表推导式[token for line in tokens for token in line]中：
- 1.外层循环：for line in tokens 会遍历外层列表tokens的每个元素，这里的line代表外层列表中的每个子列表
- 2.内层循环：for token in line 会遍历每个子列表line中的元素，这个的token代表子列表中的每个具体元素（词元）
- 3.表达式：token是列表推导式生成新列表时的元素，它会将内层循环遍历到的每个token依次添加到新生成的列表中
测算代码如下：

In [5]:
# tokens1 = [
#     ["apple", "banana"],
#     ["cherry", "date"],
#     ["elderberry"]
# ]
# flattened_tokens = [token for line in tokens1 for token in line]
# print(flattened_tokens)

In [6]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


在上述代码中，我们打印了前几个高频词元及索引

现在，我们将每一个文本行转换成一个数字索引列表

In [7]:
for i in [0,10]:
    print('文本：',tokens[i])
    print('索引：',vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引： [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 8.2.4 整合所有功能
将所有功能打包到load_corpus_time_machine函数中

有两项改变：
- 使用字符（而不是单词）实现文本词元化
- 每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的corpus仅处理为单个列表，而不是使用多词元列表构成的一个列表

In [8]:
def load_corpus_time_machine(max_tokens=-1): #@save
    '''
    :param max_token:
    :return: 返回时光机器数据集的词元索引列表和词表
    '''
    lines = read_time_machine()
    tokens = tokenize(lines,'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子
    # 或一个段落，所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in lines]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = load_corpus_time_machine()
len(corpus),len(vocab)

(10374841, 4580)

AttributeError: module 'd2l.torch' has no attribute 'read_time_machine'

个人理解：
vocab是一个字典；corpus是一个列表